In [1]:
import json
import re
import glob

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#got from https://medium.com/the-artificial-impostor/detecting-chinese-characters-in-unicode-strings-4ac839ba313a
#detect whether there is Han character in word
def detectCJK(word):
    if re.search("[\u4e00-\u9FFF]", word):
        return True
    return False

In [4]:
#find whether the word is in the list
def containsWord(word, lst):
    return word.lower() in [elem.lower() for elem in lst]

In [5]:
#find whether the same word, translation pair is in the other dict
def hasTranslation(word, translation,  dct):
    #remove weird square brackets from wikt2dict
    translation = translation.replace("[","").replace("]","")

    #ignoring Chinese characters
    if detectCJK(translation): #detect whether there are any Chinese characters in there
        return False

    #the translation is also in the other dict for the word
    elif(containsWord(translation, dct[word])):
        return True
    else:
        return False

In [6]:
dictFiles = glob.glob("/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/*")
print(dictFiles)
pathsDict = {file:index for (index, file) in enumerate(dictFiles)}
print(pathsDict)

['/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/soha hu-fr-vi.json', '/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/soha hu-en-vi.json', '/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/vdict hu-en-vi.json', '/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/wikt hu-en-vi.json', '/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/wikt hu-fr-vi.json', '/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/vdict hu-fr-vi.json']
{'/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/soha hu-fr-vi.json': 0, '/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/soha hu-en-vi.json': 1, '/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/vdict hu-en-vi.json': 2, '/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Triangulate Paths/wikt hu

In [7]:
for i in range(0, len(dictFiles)):
    #use one source as the initial dictionary
    initialFile = dictFiles[i]
    otherFiles = dictFiles[:i] + dictFiles[i+1:]
    
    with open(initialFile, "r") as infile:
        initialDict = json.load(infile)
    
    initialName = initialFile.split("/")[-1]
    print("The intial dict is {}".format(initialName))
    
    otherDicts = {}
    for file in otherFiles:
        with open(file, "r") as infile:
            otherDict = json.load(infile)
        print("Reading in {}".format(file.split("/")[-1]))
        otherDicts[file] = otherDict

    outputPath = "/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/Confirmed by Paths/" + "confirmed " + initialName.split(".")[0]
    print("Outputting in {}".format(outputPath))
    outfile = open(outputPath, "w")
    for (word, translations) in initialDict.items():
        for translation in translations:
            occurCount = 1
            occurIn = [pathsDict[initialFile]]
            #checking if any of the translation exist in another dict
            for file in otherFiles:
                otherDict = otherDicts[file]
                if (word in otherDict and hasTranslation(word, translation, otherDict)):
                    occurCount += 1
                    occurIn.append(pathsDict[file])
                    
            if occurCount > 1:
                conf = occurCount/6
                outfile.write(word + "\t" + translation + "\t" + translation + "\t" + str(occurIn) + "\t" + str(conf) + "\n")
                print("Word {} is translated to {}, and it occurs in {}, with a confidence score of {}".format(word, translation, occurIn, conf))
    outfile.close()

Streaming output truncated to the last 5000 lines.
Word kötelezi magát is translated to mất, and it occurs in [5, 0], with a confidence score of 0.3333333333333333
Word kötelezi magát is translated to choán, and it occurs in [5, 0], with a confidence score of 0.3333333333333333
Word kötelezi magát is translated to uống, and it occurs in [5, 0], with a confidence score of 0.3333333333333333
Word kötelezi magát is translated to chiếm, and it occurs in [5, 0], with a confidence score of 0.3333333333333333
Word kötelezi magát is translated to xách, and it occurs in [5, 0], with a confidence score of 0.3333333333333333
Word kötelezi magát is translated to đón, and it occurs in [5, 0], with a confidence score of 0.3333333333333333
Word kötelezi magát is translated to dính, and it occurs in [5, 0], with a confidence score of 0.3333333333333333
Word kötelezi magát is translated to bỏ, and it occurs in [5, 0], with a confidence score of 0.3333333333333333
Word kötelezi magát is translated to th